In [70]:
# Display the current working directory
getwd();
# If necessary, change the path below to the directory where the data files are stored.
# "." means current directory. On Windows use a forward slash / instead of the usual \.
workingDir = "/home/jovyan/work/phd/datasets/unbalanced/";
setwd(workingDir);
# Load WGCNA package
library(WGCNA)
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);

[1] "/home/jovyan/work/phd/datasets/unbalanced"

In [71]:
add_legend <- function(...) {
  opar <- par(fig=c(0, 1, 0, 1), oma=c(0, 0, 0, 0), 
    mar=c(0, 0, 0, 0), new=TRUE)
  on.exit(par(opar))
  plot(0, 0, type='n', bty='n', xaxt='n', yaxt='n')
  legend(...)
}

In [96]:
datExpr <- t(read.csv("mainTable.csv", check.names=FALSE, row.names = "ensg"))
genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples
allTraits <- read.csv("files.dat")

In [5]:
log1 <- function(x){
    log(x+1)
}
datExpr <- as.data.frame(apply(datExpr,2, log2))
dimnames(datExpr)[[1]] <- samples

ERROR: Error in apply(datExpr, 2, log1): object 'datExpr' not found


In [97]:
datExpr[1:10,1:5]

,ENSG00000172179,ENSG00000170477,ENSG00000171401,ENSG00000115138,ENSG00000163209
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-1117F-2426-SM-5EGGH,430.52410,205.93553,369.1534,423.18932,238.3773
GTEX-111CU-0926-SM-5EGIK,301.79219,180.69427,315.7691,424.26025,191.2650
GTEX-111YS-1126-SM-5GZYQ,9.44658,152.65697,197.3795,34.49421,114.8609
GTEX-1122O-1926-SM-5EGIQ,273.11832,152.70703,290.6506,371.47950,207.1419
GTEX-113JC-2226-SM-5EGJG,287.49936,83.05799,133.7996,306.57446,110.5678
GTEX-117YW-2226-SM-5N9DB,385.89756,190.95811,233.6855,195.97702,133.7426
GTEX-117YX-1026-SM-5H11V,328.31810,273.87678,302.9630,308.09987,230.9467
GTEX-11DXX-1326-SM-5GIDZ,16.77433,173.68213,276.4686,144.46671,192.5999
GTEX-11DXX-1526-SM-5H115,363.22682,399.18619,574.7119,411.27001,331.2084


In [98]:
head(allTraits)

SAMPID,SMTS,SMTSD
<chr>,<chr>,<chr>
GTEX-1117F-2426-SM-5EGGH,Uterus,Uterus
GTEX-111CU-0926-SM-5EGIK,Stomach,Stomach
GTEX-111YS-1126-SM-5GZYQ,Stomach,Stomach
GTEX-1122O-1926-SM-5EGIQ,Stomach,Stomach
GTEX-113JC-2226-SM-5EGJG,Uterus,Uterus
GTEX-117YW-2226-SM-5N9DB,Stomach,Stomach


In [99]:
#labels <- c('primary_site', 'malignacy', 'disease_type', 'stage')
#labels <- c('cases.0.project.primary_site', 'cases.0.project.disease_type')
labels <- c('SMTS', 'SMTSD')
datTraits <- allTraits[,labels]
rownames(datTraits) <- allTraits$SAMPID
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
GTEX-1117F-2426-SM-5EGGH,Uterus,Uterus
GTEX-111CU-0926-SM-5EGIK,Stomach,Stomach
GTEX-111YS-1126-SM-5GZYQ,Stomach,Stomach
GTEX-1122O-1926-SM-5EGIQ,Stomach,Stomach
GTEX-113JC-2226-SM-5EGJG,Uterus,Uterus
GTEX-117YW-2226-SM-5N9DB,Stomach,Stomach


In [100]:
table( dimnames(datExpr)[[1]]==datTraits$file_name)

< table of extent 0 >

In [101]:
y = datTraits$primary_site

In [102]:
meanExpressionBySample=apply( datExpr,1,mean, na.rm=T)
NumberMissingBySample=apply( is.na(data.frame(datExpr)),1, sum)

## 1

In [103]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1


[1] TRUE

In [104]:
sampleTree = hclust(dist(datExpr), method = "average");
# Plot the sample tree: Open a graphic output window of size 12 by 9 inches
# The user should change the dimensions if the window is too large or too small.
sizeGrWindow(12,9)
svg("samplehier.svg")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
cex.axis = 1.5, cex.main = 2)
dev.off()

png 
  2

In [82]:
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 80, minSize = 10)
table(clust)

clust
  0 
590 

In [83]:
# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr = datExpr[keepSamples, ]
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples

In [94]:
head(datExpr)

,ENSG00000172179,ENSG00000170477,ENSG00000171401,ENSG00000115138,ENSG00000163209,ENSG00000126233,ENSG00000172016,ENSG00000143536,ENSG00000205420,ENSG00000259384,⋯,ENSG00000151640,ENSG00000125966,ENSG00000160221,ENSG00000146677,ENSG00000023445,ENSG00000164442,ENSG00000107968,ENSG00000170776,ENSG00000112699,ENSG00000058668
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-1117F-2426-SM-5EGGH,430.52410,205.93553,369.1534,423.18932,238.3773,67.43703,231.1503,94.50833,253.0820,500.5849,⋯,238.73625,144.04207,283.410440,415.8660,274.7387,706.6089,463.9232,658.1502,303.1924,824.3174
GTEX-111CU-0926-SM-5EGIK,301.79219,180.69427,315.7691,424.26025,191.2650,54.26048,493.4988,129.27817,232.3946,427.2023,⋯,141.14262,66.52108,25.374728,316.3982,439.7803,570.3765,245.6543,413.7504,596.8782,300.3242
GTEX-111YS-1126-SM-5GZYQ,9.44658,152.65697,197.3795,34.49421,114.8609,42.63184,245.4965,49.82509,123.8175,0.0000,⋯,104.33445,47.25917,24.634728,193.6591,394.0167,497.6364,279.7428,455.8880,505.6150,420.1634
GTEX-1122O-1926-SM-5EGIQ,273.11832,152.70703,290.6506,371.47950,207.1419,43.00711,254.8930,108.94982,194.3359,408.5765,⋯,83.03365,67.27421,236.709145,256.4134,402.6800,494.0637,237.6012,366.5620,457.9542,254.9423
GTEX-113JC-2226-SM-5EGJG,287.49936,83.05799,133.7996,306.57446,110.5678,92.69862,0.0000,44.73147,115.5102,447.3137,⋯,348.41381,78.30794,28.321062,336.7511,421.8781,870.5978,555.4896,609.7611,327.9174,777.1489
GTEX-117YW-2226-SM-5N9DB,385.89756,190.95811,233.6855,195.97702,133.7426,67.57256,383.3902,72.29031,120.7268,261.1645,⋯,117.31274,56.34510,8.626746,267.6042,652.7477,529.3885,534.7666,536.2470,540.8032,556.5293


In [95]:
rownames(allTraits) <- allTraits$SAMPID
datTraits <- allTraits[keepSamples,labels]
rownames(datTraits) <- rownames(allTraits[keepSamples,labels])
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
NA,NA,NA
NA.1,NA,NA
NA.2,NA,NA
NA.3,NA,NA
NA.4,NA,NA
NA.5,NA,NA


In [32]:
collectGarbage();

In [33]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr), method = "average")
fontsize=.5
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("samplehiercut.pdf")
plotDendroAndColors(sampleTree2, traitColors,
    groupLabels = labels,
    main = "Sample dendrogram and trait heatmap",
    dendroLabels=FALSE,
    rowText=datTraits,
    rowTextIgnore=labels[[2]]
    )
dev.off()

png 
  2

## 2

In [105]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to=20, by=2))
# Call the network topology analysis function
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
# Plot the results:
sizeGrWindow(9, 5)
pdf("scaletopology.pdf")
par(mfrow = c(1,2));
cex1 = 0.9;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pickSoftThreshold: will use block size 1000.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 1000 of 1000
   Power SFT.R.sq  slope truncated.R.sq mean.k. median.k. max.k.
1      1    0.823  0.548          0.811  278.00   278.000  465.0
2      2    0.562 -0.252          0.632  129.00   115.000  297.0
3      3    0.841 -0.583          0.835   74.20    58.900  214.0
4      4    0.872 -0.734          0.876   47.80    32.700  165.0
5      5    0.893 -0.847          0.905   33.10    19.000  132.0
6      6    0.911 -0.917          0.932   24.00    12.000  108.0
7      7    0.911 -0.980          0.940   18.00     8.150   90.5
8      8    0.914 -1.020          0.946   13.90     5.570   76.9
9      9    0.900 -1.070          0.948   11.00     3.830   66.0
10    10    0.890 -1.100          0.933    8.79     2.830   57.3
11    12    0.863 -1.190          0.926    5.88     1.450   44.1
12    14    0.897 -1.220          0.952    4.09     0.894   34.8


png 
  2

In [106]:
net = blockwiseModules(datExpr, power = 6,
TOMType = "unsigned", minModuleSize = 30,
reassignThreshold = 0, mergeCutHeight = 0.15,
numericLabels = TRUE, pamRespectsDendro = FALSE,
saveTOMs = TRUE,
saveTOMFileBase = "wgcna",
verbose = 3)

 Calculating module eigengenes block-wise from all genes
   Flagging genes and samples with too many missing values...
    ..step 1
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will not use multithreading.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file wgcna-block.1.RData
 ....clustering..
 ....detecting modules..
 ....calculating module eigengenes..
 ....checking kME in modules..
     ..removing 13 genes from module 1 because their KME is too low.
     ..removing 2 genes from module 2 because their KME is too low.
     ..removing 1 genes from module 3 because their KME is too low.
     ..removing 1 genes from module 4 because their KME is too low.
 ..merging modules that are too close..
     mergeCloseModules: Merging modules whose distance is less than 0.15
       Calculating new MEs...


In [107]:
# open a graphics window
sizeGrWindow(12, 9)
pdf("netdendograms.pdf")
# Convert labels to colors for plotting
mergedColors = labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)
dev.off()

png 
  2

In [108]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]];

## 3

In [46]:
datTraits$cases.0.project.primary_site <- match(datTraits$cases.0.project.primary_site, unique(datTraits$cases.0.project.primary_site))
datTraits$cases.0.project.disease_type <- match(datTraits$cases.0.project.disease_type, unique(datTraits$cases.0.project.disease_type))

In [109]:
datTraits$SMTS <- match(datTraits$SMTS, unique(datTraits$SMTS))
datTraits$SMTSD <- match(datTraits$SMTSD, unique(datTraits$SMTSD))

In [40]:
datTraits$primary_site <- match(datTraits$primary_site, unique(datTraits$primary_site))
#datTraits$secondary_site <- match(datTraits$secondary_site, unique(datTraits$secondary_site))
datTraits$malignacy <- match(datTraits$malignacy, unique(datTraits$malignacy))
datTraits$disease_type <- match(datTraits$disease_type, unique(datTraits$disease_type))
datTraits$stage <- match(datTraits$stage, unique(datTraits$stage))

ERROR: Error in `$<-.data.frame`(`*tmp*`, primary_site, value = integer(0)): replacement has 0 rows, data has 1000


In [110]:
# Define numbers of genes and samples
nGenes = ncol(datExpr);
nSamples = nrow(datExpr);
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

In [111]:
pdf("moduleTrait.pdf", width=9, height=18)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
xLabels = labels,
yLabels = names(MEs),
ySymbols = names(MEs),
colorLabels = FALSE,
colors = blueWhiteRed(50),
textMatrix = textMatrix,
setStdMargins = FALSE,
cex.text = 0.5,
zlim = c(-1,1),
main = paste("Module-trait relationships"))
dev.off()

png 
  2

In [112]:
# Define variable weight containing the weight column of datTrait
weight = as.data.frame(datTraits$SMTS);
names(weight) = "weight"
# names (colors) of the modules
modNames = substring(names(MEs), 3)
geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"));
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples));
names(geneModuleMembership) = paste("MM", modNames, sep="");
names(MMPvalue) = paste("p.MM", modNames, sep="");
geneTraitSignificance = as.data.frame(cor(datExpr, weight, use = "p"));
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples));
names(geneTraitSignificance) = paste("GS.", names(weight), sep="");
names(GSPvalue) = paste("p.GS.", names(weight), sep="");

In [113]:
module = "yellow"
column = match(module, modNames);
moduleGenes = moduleColors==module;
sizeGrWindow(7, 7);
pdf("modulemembership.pdf")
par(mfrow = c(1,1));
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
    abs(geneTraitSignificance[moduleGenes, 1]),
    xlab = paste("Module Membership in", module, "module"),
    ylab = "Gene significance for primary_site",
    main = paste("Module membership vs. gene significance\n"),
    cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

png 
  2

In [114]:
# Create the starting data frame
geneInfo0 = data.frame(
moduleColor = moduleColors,
geneTraitSignificance,
GSPvalue)
# Order modules by their significance for weight
modOrder = order(-abs(cor(MEs, weight, use = "p")));
# Add module membership information in the chosen order
for (mod in 1:ncol(geneModuleMembership))
{
oldNames = names(geneInfo0)
geneInfo0 = data.frame(geneInfo0, geneModuleMembership[, modOrder[mod]],
MMPvalue[, modOrder[mod]]);
names(geneInfo0) = c(oldNames, paste("MM.", modNames[modOrder[mod]], sep=""),
paste("p.MM.", modNames[modOrder[mod]], sep=""))
}
# Order the genes in the geneInfo variable first by module color, then by geneTraitSignificance
geneOrder = order(geneInfo0$moduleColor, -abs(geneInfo0$GS.weight));
geneInfo = geneInfo0[geneOrder, ]
write.csv(geneInfo, file = "geneInfo.csv")

In [115]:
# Read in the probe annotation
annot = read.csv(file = "geneInfo.csv");
# Match probes in the data set to the probe IDs in the annotation file
probes = names(datExpr)
allLLIDs = probes
# $ Choose interesting modules
intModules = c("greenyellow", "darkorange", "darkgreen")
for (module in intModules)
{
# Select module probes
modGenes = (moduleColors==module)
# Get their entrez ID codes
modLLIDs = probes[modGenes];
# Write them into a file
fileName = paste("LocusLinkIDs-", module, ".txt", sep="");
write.table(as.data.frame(modLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)
}
# As background in the enrichment analysis, we will use all probes in the analysis.
fileName = paste("LocusLinkIDs-all.txt", sep="");
write.table(as.data.frame(allLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)

In [121]:
require("biomaRt")
mart <- useMart("ENSEMBL_MART_ENSEMBL")
mart <- useDataset("hsapiens_gene_ensembl", mart)

ens <- allLLIDs
ensLookup <- gsub("\\.[0-9]*$", "", ens)

annotLookup <- getBM(
  mart=mart,
  attributes=c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"),
  filter="ensembl_gene_id",
  values=ensLookup,
  uniqueRows=TRUE)

annotLookup <- data.frame(
  ens[match(annotLookup$ensembl_gene_id, ensLookup)],
  annotLookup)

colnames(annotLookup) <- c(
  "original_id",
  c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"))

annotLookup

Loading required package: biomaRt


ERROR: Error in .processResults(postRes, mart = mart, sep = sep, fullXmlQuery = fullXmlQuery, : Query ERROR: caught BioMart::Exception::Database: Could not connect to mysql database ensembl_mart_99: DBI connect('database=ensembl_mart_99;host=127.0.0.1;port=5316','ensro',...) failed: Can't connect to MySQL server on '127.0.0.1' (111) at /net/isilonP/public/ro/ensweb/live/mart/www_99/biomart-perl/lib/BioMart/Configuration/DBLocation.pm line 98.



## 5 

In [116]:
# Re-cluster samples
sampleTree3 = hclust(dist(MEs0), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
svg("nethier.svg", width=10)
# Set the plot margin: bottom, left, top & right
par(mar = c(5, 4, 1.4, 0.2) + 0.1,
    xpd = NA) # allow content to go into outer margin 
plotDendroAndColors(sampleTree3, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE,
#rowText=allTraits[keepSamples,labels],
#rowTextIgnore=labels[[2]],
addGuide=TRUE)
dev.off()

png 
  2

In [138]:
rownames(allTraits)=allTraits$SAMPID
unique(allTraits[rownames(datTraits),'SMTS'])

[1] "Uterus"  "Stomach" "Kidney"

In [139]:
svg("nethier_legend.svg", width=10)
add_legend("topleft", legend = unique(allTraits[rownames(datTraits),'SMTS']), pch = 1, pt.cex = 1, cex = 1.5, bty = 'n', ncol=2,
       inset = c(0.1, 0.), # place outside
       title = "SMTS", 
       col = traitColors)
dev.off()

Warning message in par(fig = c(0, 1, 0, 1), oma = c(0, 0, 0, 0), mar = c(0, 0, 0, :
“calling par(new=TRUE) with no plot”

png 
  2

In [140]:
collectGarbage()

In [61]:
clust = cutreeStatic(sampleTree3, cutHeight = 0.4, minSize = 10)
table(clust)

clust
  1 
590 

In [62]:
# Re-cluster samples after cut
sampleTree4 = hclust(dist(MEs0[(clust==1),]), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits[(clust==1),]);
# Plot the sample dendrogram and the colors underneath.
svg("nethiercut.svg")
plotDendroAndColors(sampleTree4, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE)
dev.off()

png 
  2

In [63]:
# Calculate topological overlap anew: this could be done more efficiently by saving the TOM
# calculated during module detection, but let us do it again here.
dissTOM = 1-TOMsimilarityFromExpr(datExpr, power = 6);
# Transform dissTOM with a power to make moderately strong connections more visible in the heatmap
plotTOM = dissTOM^7;
# Set diagonal to NA for a nicer plot
diag(plotTOM) = NA;
# Call the plot function
sizeGrWindow(9,9)
svg("networkmap.svg")
TOMplot(plotTOM, geneTree, moduleColors, main = "Network heatmap plot, all genes")
dev.off()

TOM calculation: adjacency..
..will not use multithreading.
 Fraction of slow calculations: 0.000000
..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.


png 
  2

In [64]:
#selct genes to be faster
nSelect = nGenes*0.7
# For reproducibility, we set the random seed
set.seed(10);
select = sample(nGenes, size = nSelect);
selectTOM = dissTOM[select, select];
# There’s no simple way of restricting a clustering tree to a subset of genes, so we must re-cluster.
selectTree = hclust(as.dist(selectTOM), method = "average")
selectColors = moduleColors[select];
# Open a graphical window
sizeGrWindow(9,9)
svg("networkmap.svg")
# Taking the dissimilarity to a power, say 10, makes the plot more informative by effectively changing
# the color palette; setting the diagonal to NA also improves the clarity of the plot
plotDiss = selectTOM^7;
diag(plotDiss) = NA;
TOMplot(plotDiss, selectTree, selectColors, main = "Network heatmap plot, selected genes")
dev.off()

png 
  2

In [60]:
# Recalculate module eigengenes
MEs = moduleEigengenes(datExpr, moduleColors)$eigengenes
# Isolate weight from the clinical traits
weight = as.data.frame(datTraits$SMTS);
names(weight) = "SMTS"
# Add the weight to existing module eigengenes
MET = orderMEs(cbind(MEs, weight))
# Plot the relationships among the eigengenes and the trait
sizeGrWindow(5,7.5);
svg("eigenHeat.svg")
par(cex = 0.9)
plotEigengeneNetworks(MET, "", marDendro = c(0,4,1,2), marHeatmap = c(3,4,1,2), cex.lab = 0.8, xLabelsAngle = 90)
dev.off()

ERROR: Error in names(weight) = "SMTS": 'names' attribute [1] must be the same length as the vector [0]


In [ ]:
# Plot the dendrogram
sizeGrWindow(6,6);
svg("eigendendogram.svg")
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene dendrogram", marDendro = c(0,4,2,0),
plotHeatmaps = FALSE)
# Plot the heatmap matrix (note: this plot will overwrite the dendrogram plot)
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene adjacency heatmap", marHeatmap = c(3,4,2,2),
plotDendrograms = FALSE, xLabelsAngle = 90)
dev.off()

In [67]:
write.csv(cutree(sampleTree3, k=1), "wgcna/wgcna_level_2_labels.csv")

In [144]:
write.csv(geneModuleMembership, "wgcna/wgcna_level_0_word-dist.csv")

In [143]:
write.csv(MEs, "wgcna/wgcna_level_0_topic-dist.csv")